# Data Audit

This script takes an audit of the EPC data for England. It looks at how many missing values each feature has, how many unique values the categorical features have then applies preprocessing steps to remove outliers and drop features with too many missing values.

In [1]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt

In [9]:
# set variables from config file
config_path = os.path.abspath('..')

with open(config_path + '/config-example copy.json', 'r') as f:
    config = json.load(f)

processing_path = config['DEFAULT']['processing_path']
epc_england_fname = config['DEFAULT']['epc_england_fname']
epc_train_fname = config['DEFAULT']['epc_train_fname']
epc_test_fname = config['DEFAULT']['epc_test_fname']
epc_train_clean_fname = config['DEFAULT']['epc_train_clean_fname']
epc_test_clean_fname = config['DEFAULT']['epc_test_clean_fname']
epc_fname_suffix = config['DEFAULT']['epc_fname_suffix']

In [10]:
#Specify ther import dictionary for the dates
dtype_dict = {'INSPECTION_DATE':'str'}

epc_train = pd.read_csv(os.path.join(processing_path,epc_train_fname) + epc_fname_suffix, header = 0, delimiter = ',', 
                       dtype = dtype_dict, parse_dates = ['INSPECTION_DATE'])

epc_test = pd.read_csv(os.path.join(processing_path,epc_test_fname) + epc_fname_suffix,header = 0, delimiter = ',',
                        dtype = dtype_dict, parse_dates = ['INSPECTION_DATE'])

/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_73614/2719602185.py:4: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  epc_train = pd.read_csv(os.path.join(processing_path,epc_train_fname) + epc_fname_suffix, header = 0, delimiter = ',',
/var/folders/ld/qgzpl61x5_91yljdqrrpz9g00000gn/T/ipykernel_73614/2719602185.py:7: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  epc_test = pd.read_csv(os.path.join(processing_path,epc_test_fname) + epc_fname_suffix,header = 0, delimiter = ',',


In [11]:
epc_train['inspection_year'] = epc_train['INSPECTION_DATE'].dt.year

In [12]:
epc_train['inspection_year'].value_counts(normalize=True)

2009    8.936362e-02
2014    8.821498e-02
2013    8.141831e-02
2015    7.656052e-02
2010    7.315055e-02
2021    7.132493e-02
2019    6.745494e-02
2011    6.580542e-02
2020    6.478116e-02
2012    6.344867e-02
2016    6.095409e-02
2018    6.072757e-02
2022    5.726744e-02
2017    5.128040e-02
2008    2.811319e-02
2007    1.083474e-04
2006    1.784242e-05
2004    4.395958e-06
2000    1.034343e-06
2002    7.757573e-07
2005    5.171715e-07
1989    5.171715e-07
2001    2.585858e-07
1988    2.585858e-07
2003    2.585858e-07
Name: inspection_year, dtype: float64

In [13]:
print(len(epc_train))
print(len(epc_test))

3867189
966798


### Investigate categoric and numeric fields

In [14]:
# Get numeric and categorical variable names 
var_list_num = epc_train.select_dtypes(include= 'number').columns.tolist()

var_list_cat = epc_train.select_dtypes(include= ['object','category']).columns.tolist()
var_list_cat.remove('LMK_KEY')
var_list_cat.remove('POSTCODE')

In [15]:
# Percentage of missing values in numeric fields
epc_train[var_list_num].isnull().sum().sort_values(ascending = False) / epc_train.shape[0]

SHEATING_ENV_EFF                 1.000000
SHEATING_ENERGY_EFF              1.000000
FLAT_STOREY_COUNT                0.943796
UNHEATED_CORRIDOR_LENGTH         0.897037
FLOOR_HEIGHT                     0.496401
EXTENSION_COUNT                  0.127270
NUMBER_HEATED_ROOMS              0.127270
NUMBER_HABITABLE_ROOMS           0.123416
MULTI_GLAZE_PROPORTION           0.109363
LOW_ENERGY_LIGHTING              0.042368
NUMBER_OPEN_FIREPLACES           0.024469
BUILDING_REFERENCE_NUMBER        0.000000
CURRENT_ENERGY_EFFICIENCY        0.000000
TOTAL_FLOOR_AREA                 0.000000
HOT_WATER_COST_CURRENT           0.000000
HEATING_COST_CURRENT             0.000000
LIGHTING_COST_CURRENT            0.000000
CO2_EMISS_CURR_PER_FLOOR_AREA    0.000000
CO2_EMISSIONS_CURRENT            0.000000
ENERGY_CONSUMPTION_CURRENT       0.000000
inspection_year                  0.000000
dtype: float64

Drop SHEATING_ENV_EFF, SHEATING_ENERGY_EFF, FLAT_STOREY_COUNT AND UNHEATED_CORRIDOR_LENGTH due to high percentage of 
missing values. Keep FLOOR_HEIGHT but be careful

In [16]:
audit_num = epc_train[var_list_num].describe(percentiles = [0.25,0.5,0.75,0.9,0.95,0.99])
audit_num

,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_EFFICIENCY,ENERGY_CONSUMPTION_CURRENT,CO2_EMISSIONS_CURRENT,CO2_EMISS_CURR_PER_FLOOR_AREA,LIGHTING_COST_CURRENT,HEATING_COST_CURRENT,HOT_WATER_COST_CURRENT,TOTAL_FLOOR_AREA,FLAT_STOREY_COUNT,...,EXTENSION_COUNT,NUMBER_HABITABLE_ROOMS,NUMBER_HEATED_ROOMS,LOW_ENERGY_LIGHTING,NUMBER_OPEN_FIREPLACES,SHEATING_ENERGY_EFF,SHEATING_ENV_EFF,UNHEATED_CORRIDOR_LENGTH,FLOOR_HEIGHT,inspection_year
count,3.867189e+06,3.867189e+06,3.867189e+06,3.867189e+06,3.867189e+06,3.867189e+06,3.867189e+06,3.867189e+06,3.867189e+06,217353.000000,...,3.375013e+06,3.389917e+06,3.375013e+06,3.703344e+06,3.772562e+06,0.0,0.0,398176.000000,1.947512e+06,3.867189e+06
mean,5.706386e+09,6.391303e+01,2.614394e+02,4.111396e+00,4.656929e+01,7.877851e+01,6.740487e+02,1.408185e+02,9.102566e+01,2.942071,...,4.695469e-01,4.382142e+00,4.287525e+00,5.530258e+01,1.458221e-01,NaN,NaN,5.674900,2.422597e+00,2.014964e+03
std,3.254443e+09,1.456409e+01,1.404945e+02,3.465071e+00,2.482068e+01,4.050691e+01,5.654613e+02,7.797825e+01,6.301891e+01,1.476704,...,7.512637e-01,1.856126e+00,1.860881e+00,3.656648e+01,4.829753e-01,NaN,NaN,4.329554,1.512956e+00,4.165944e+00
min,2.000000e+00,0.000000e+00,-3.774000e+03,-4.080000e+01,-6.810000e+02,-8.000000e+00,-2.730000e+02,-5.460000e+02,0.000000e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,NaN,NaN,0.000000,0.000000e+00,1.988000e+03
25%,2.911689e+09,5.600000e+01,1.780000e+02,2.200000e+00,3.200000e+01,5.400000e+01,3.530000e+02,9.200000e+01,6.100000e+01,2.000000,...,0.000000e+00,3.000000e+00,3.000000e+00,2.100000e+01,0.000000e+00,NaN,NaN,3.840000,2.320000e+00,2.011000e+03
50%,5.819712e+09,6.600000e+01,2.390000e+02,3.400000e+00,4.300000e+01,7.200000e+01,5.500000e+02,1.160000e+02,8.000000e+01,3.000000,...,0.000000e+00,4.000000e+00,4.000000e+00,5.700000e+01,0.000000e+00,NaN,NaN,5.670000,2.400000e+00,2.015000e+03
75%,8.731975e+09,7.300000e+01,3.180000e+02,5.000000e+00,5.700000e+01,9.500000e+01,8.180000e+02,1.630000e+02,1.040000e+02,3.000000,...,1.000000e+00,5.000000e+00,5.000000e+00,1.000000e+02,0.000000e+00,NaN,NaN,7.300000,2.460000e+00,2.019000e+03
90%,1.000099e+10,8.200000e+01,4.230000e+02,7.300000e+00,7.500000e+01,1.230000e+02,1.195000e+03,2.370000e+02,1.440000e+02,4.000000,...,1.000000e+00,7.000000e+00,7.000000e+00,1.000000e+02,1.000000e+00,NaN,NaN,9.170000,2.620000e+00,2.021000e+03
95%,1.000216e+10,8.400000e+01,5.110000e+02,9.300000e+00,9.000000e+01,1.450000e+02,1.556000e+03,2.910000e+02,1.810000e+02,5.000000,...,2.000000e+00,8.000000e+00,8.000000e+00,1.000000e+02,1.000000e+00,NaN,NaN,10.600000,2.770000e+00,2.022000e+03
99%,1.000313e+10,8.700000e+01,7.530000e+02,1.600000e+01,1.300000e+02,2.060000e+02,2.747000e+03,4.320000e+02,2.935424e+02,9.000000,...,3.000000e+00,1.000000e+01,1.000000e+01,1.000000e+02,2.000000e+00,NaN,NaN,14.720000,3.120000e+00,2.022000e+03


We have some negative values and some massively large

Clip all below 0

Clip all at the 95th percentile with the exception of the following

In [17]:
# Percentage of missing values in categoric fields
epc_train[var_list_cat].isnull().sum().sort_values(ascending = False) / epc_train.shape[0]

FLOOR_ENV_EFF               0.897410
FLOOR_ENERGY_EFF            0.897410
HEAT_LOSS_CORRIDOR          0.793382
FLAT_TOP_STOREY             0.768570
FLOOR_LEVEL                 0.755313
SOLAR_WATER_HEATING_FLAG    0.378019
MAIN_HEATING_CONTROLS       0.249966
GLAZED_TYPE                 0.172452
ROOF_ENV_EFF                0.136582
ROOF_ENERGY_EFF             0.136582
MECHANICAL_VENTILATION      0.127270
MAINS_GAS_FLAG              0.124486
ENERGY_TARIFF               0.090700
BUILT_FORM                  0.019089
MAIN_FUEL                   0.006831
HOT_WATER_ENV_EFF           0.003158
MAINHEAT_ENERGY_EFF         0.003139
MAINHEAT_ENV_EFF            0.003121
HOT_WATER_ENERGY_EFF        0.003118
MAINHEATC_ENV_EFF           0.002655
MAINHEATC_ENERGY_EFF        0.002655
SECONDHEAT_DESCRIPTION      0.002648
MAINHEATCONT_DESCRIPTION    0.002637
HOTWATER_DESCRIPTION        0.002637
LIGHTING_ENERGY_EFF         0.001809
WINDOWS_DESCRIPTION         0.001761
ROOF_DESCRIPTION            0.001583
W

Drop FLOOR_ENV_EFF, FLOOR_ENERGY_EFF and FLAT_TOP_STOREY due to percentage of missing values

In [18]:
audit_cat = epc_train[var_list_cat].describe()
epc_train[var_list_cat].nunique().sort_values(ascending = False)

LODGEMENT_DATE              5111
WALLS_DESCRIPTION           1308
FLOOR_DESCRIPTION            856
ROOF_DESCRIPTION             726
LIGHTING_DESCRIPTION         354
MAIN_HEATING_CONTROLS        176
HOTWATER_DESCRIPTION         155
region                       111
MAINHEATCONT_DESCRIPTION      80
FLOOR_LEVEL                   73
SECONDHEAT_DESCRIPTION        66
MAIN_FUEL                     57
WINDOWS_DESCRIPTION           55
COUNTY                        24
TRANSACTION_TYPE              18
ENERGY_TARIFF                  9
GLAZED_TYPE                    8
CURRENT_ENERGY_RATING          7
BUILT_FORM                     6
LIGHTING_ENV_EFF               5
MAINHEATC_ENV_EFF              5
ROOF_ENERGY_EFF                5
MAINHEAT_ENV_EFF               5
MAINHEAT_ENERGY_EFF            5
ROOF_ENV_EFF                   5
LIGHTING_ENERGY_EFF            5
MAINHEATC_ENERGY_EFF           5
WINDOWS_ENERGY_EFF             5
WALLS_ENV_EFF                  5
WALLS_ENERGY_EFF               5
WINDOWS_EN

Drop SHEATING_ENV_EFF, SHEATING_ENERGY_EFF, FLAT_STOREY_COUNT AND UNHEATED_CORRIDOR_LENGTH due to high percentage of 
missing values. Keep FLOOR_HEIGHT but be careful

Drop FLOOR_ENV_EFF, FLOOR_ENERGY_EFF and FLAT_TOP_STOREY due to percentage of missing values

### Drop unwanted fields

In [19]:
epc_train.drop(['SHEATING_ENERGY_EFF','SHEATING_ENV_EFF','FLOOR_ENV_EFF','FLAT_STOREY_COUNT', 
                'UNHEATED_CORRIDOR_LENGTH','LODGEMENT_DATE',
                'FLAT_TOP_STOREY','FLOOR_ENERGY_EFF','SOLAR_WATER_HEATING_FLAG'],
                axis = 1,
                inplace = True)

epc_test.drop(['SHEATING_ENERGY_EFF','SHEATING_ENV_EFF','FLOOR_ENV_EFF','FLAT_STOREY_COUNT', 
                'UNHEATED_CORRIDOR_LENGTH','LODGEMENT_DATE',
                'FLAT_TOP_STOREY','FLOOR_ENERGY_EFF','SOLAR_WATER_HEATING_FLAG'],
                axis = 1,
               inplace = True)

### Clip training and test datasets

In [20]:
epc_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3867189 entries, 0 to 3867188
Data columns (total 56 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   LMK_KEY                        object        
 1   region                         object        
 2   POSTCODE                       object        
 3   BUILDING_REFERENCE_NUMBER      int64         
 4   CURRENT_ENERGY_RATING          object        
 5   CURRENT_ENERGY_EFFICIENCY      int64         
 6   PROPERTY_TYPE                  object        
 7   BUILT_FORM                     object        
 8   INSPECTION_DATE                datetime64[ns]
 9   COUNTY                         object        
 10  TRANSACTION_TYPE               object        
 11  ENERGY_CONSUMPTION_CURRENT     int64         
 12  CO2_EMISSIONS_CURRENT          float64       
 13  CO2_EMISS_CURR_PER_FLOOR_AREA  float64       
 14  LIGHTING_COST_CURRENT          float64       
 15  HEATING_COST_CU

In [21]:
def call_clip(df):
    
    ''' 
    Clips the variables of the EPC data between 0 and a specified upper amount
    Parameters
      df: a dataframe containing EPC data specified below
    Returns the dataframe with clipped values
    '''
    
    df['CURRENT_ENERGY_EFFICIENCY'].clip(lower = 0.0, upper = 100, inplace = True)
    df['ENERGY_CONSUMPTION_CURRENT'].clip(lower = 0.0, upper = 750, inplace = True)
    df['CO2_EMISSIONS_CURRENT'].clip(lower = 0.0, upper = 20, inplace = True)
    df['CO2_EMISS_CURR_PER_FLOOR_AREA'].clip(lower = 0.0, upper = 100, inplace = True)
    df['LIGHTING_COST_CURRENT'].clip(lower = 0.0, upper = 210, inplace = True)
    df['HEATING_COST_CURRENT'].clip(lower = 0.0, upper = 2750, inplace = True)
    df['HOT_WATER_COST_CURRENT'].clip(lower = 0.0, upper = 430, inplace = True)
    df['TOTAL_FLOOR_AREA'].clip(lower = 0.0, upper = 290, inplace = True)
    df['MULTI_GLAZE_PROPORTION'].clip(lower = 0.0, upper = 100, inplace = True)
    df['EXTENSION_COUNT'].clip(lower = 0.0, upper = 3, inplace = True)
    df['NUMBER_HABITABLE_ROOMS'].clip(lower = 0.0, upper = 10, inplace = True)
    df['NUMBER_HEATED_ROOMS'].clip(lower = 0.0, upper = 10, inplace = True)
    df['LOW_ENERGY_LIGHTING'].clip(lower = 0.0, upper = 100, inplace = True)
    df['NUMBER_OPEN_FIREPLACES'].clip(lower = 0.0, upper = 4, inplace = True)
    df['FLOOR_HEIGHT'].clip(lower = 0.0, upper = 3, inplace = True)
    return(df)

epc_train = call_clip(epc_train)
epc_test = call_clip(epc_test)

### Export datasets

In [22]:
epc_train.to_csv(os.path.join(processing_path,epc_train_clean_fname) + epc_fname_suffix,index = False)
epc_test.to_csv(os.path.join(processing_path,epc_test_clean_fname) + epc_fname_suffix,index = False)